# Imports

In [ ]:
import tensorflow as tf
import os
import matplotlib as mp
import numpy as np
import matplotlib.pyplot as plt

from stereonet.generator import TrainingGeneratorStereoNet
from refocus_algorithms.layered_dof_tf import layered_bluring
from blur_CV_refinement.model import refnet_cv_refinement
from tf_utils import optimistic_restore
from tf_gif_summary import gif_summary

# Parameters and Flags

In [ ]:
# path to save
#TODO

full_w = 960
full_h = 540
resolution = (full_h,full_w)

# checkpoint to load
stereonet_weigts = '/home/matthieu/kgx_nfs2/ben/stereo_net_refactored/checkpts/stereoNet_ScenFlow_driving_1.7888498861e-07_step_120000.ckpt'

# checkpoint_path         = './checkpoints/training_low_lr_2e-06_step_144755val_99.ckpt'

is_from_beginning = True
n_batch = 1
n_views = 1
is_training = False

target_disparity = 128
blur_magnitude = 0.1
max_disp = 144
min_disp = 8#set by the disparity estator
downsampling_trick_max_kernel_size=11

# Parsing files

In [ ]:
n_every = 1
sanity_check=True
path_to_left_rgb = '/home/matthieu/kgx_nfs2/data/external/sceneflow/frames_cleanpass/35mm_focallength/scene_mixed_train/left/'
path_to_right_rgb = '/home/matthieu/kgx_nfs2/data/external/sceneflow/frames_cleanpass/35mm_focallength/scene_mixed_train/right'
left_rgb_names_train = [os.path.join(path_to_left_rgb,each_name) for each_name in os.listdir(path_to_left_rgb)][0::n_every]
right_rgb_names_train = [os.path.join(path_to_right_rgb,each_name) for each_name in os.listdir(path_to_right_rgb)][0::n_every]

path_to_disparity = '/home/matthieu/kgx_nfs2/data/external/sceneflow/disparity/35mm_focallength/scene_mixed_train'
disparity_names_train = [os.path.join(path_to_disparity,each_name) for each_name in os.listdir(path_to_disparity)][0::n_every]

left_rgb_names_train.sort()
right_rgb_names_train.sort()
disparity_names_train.sort()

print("sanity check started.")
n_train_data = len(left_rgb_names_train)

assert len(left_rgb_names_train) == len(right_rgb_names_train) == len(disparity_names_train), "Error : number of files doesn't match"

for idx in range(n_train_data):
    left_rgb = left_rgb_names_train[idx]
    right_rgb = right_rgb_names_train[idx]
    disparity = disparity_names_train[idx]

    assert left_rgb.split('/')[-1].split('.')[-2] == right_rgb.split('/')[-1].split('.')[-2] == disparity.split('/')[-1].split('.')[-2],"Error : wrong file name match "+left_rgb+ " "+right_rgb+ " "+disparity

    if idx % 500 == 0:
        print(idx,"out of",n_train_data,"completed")

print("sanity check finished.")

# Setting up the inputs

In [ ]:
tf.reset_default_graph()#
# training generator
training_generator = iter(TrainingGeneratorStereoNet(left_rgb_names_train, right_rgb_names_train, disparity_names_train))
generator_data_type = (tf.float32, tf.float32, tf.float32)
training_set = tf.data.Dataset.from_generator(lambda: training_generator, generator_data_type)
training_set = training_set.batch(n_batch)
buffer_size = 1
training_set = training_set.prefetch(buffer_size)
# get training iterators
training_iterator = training_set.make_initializable_iterator()
# get data placeholders
rgb_left_train, rgb_right_train, disparity_train = training_iterator.get_next()

# Sizes at compile time
rgb_left_train.set_shape(tf.TensorShape([n_batch, full_h, full_w, 3]))
rgb_right_train.set_shape(tf.TensorShape([n_batch, full_h, full_w, 3]))
disparity_train.set_shape(tf.TensorShape([n_batch, full_h, full_w, 1]))


# Network and Outputs

In [ ]:
#Setting up the stereonet
tf.logging.info("Network Declaration:")
blur_image, conf_volume, refined_conf_volume = refnet_cv_refinement(rgb_left_train, rgb_right_train, 
                                               target_disparity, blur_magnitude,
                                               downsampling_trick_max_kernel_size=downsampling_trick_max_kernel_size,
                                               min_disp = min_disp, max_disp = max_disp,
                                               is_training = True)

#Get disparity blur image
tf.logging.info("GT rebluring:")
blur_image_gt = layered_bluring(rgb_left_train, disparity_train, 
                                target_disparity,blur_magnitude, 
                                min_disp = min_disp, max_disp = max_disp,
                                downsampling_trick_max_kernel_size=downsampling_trick_max_kernel_size,
                                differenciable=False)
clipped_disp = tf.clip_by_value(disparity_train, min_disp, max_disp)
tiled_disp = tf.tile(clipped_disp, [1,1,1,18])#FIXME: hard param
tiled_slice_disp = np.tile(np.reshape(np.arange(1,18+1)*8,[1, 1, 1, -1 ]), [n_batch, full_h, full_w, 1])
cost_volume_GT = tf.exp(tf.abs(tiled_disp-tiled_slice_disp))
conf_volume_GT = tf.nn.softmax(-cost_volume_GT)

conf_volume_GT=tf.check_numerics(conf_volume_GT,    "NaN in conf_volume_GT")
blur_image_gt = tf.check_numerics(blur_image_gt,    "NaN in GT refocus image")
for d in range(18):
    refined_conf_volume[d]=tf.check_numerics( refined_conf_volume[d], "NaN in predicted conf volume")


#loss on image
loss = tf.reduce_mean(tf.pow(blur_image-blur_image_gt,2))

#loss in CV
#loss=0
#for d in range(18):
#    resized_GT_slice = tf.image.resize_bilinear(tf.expand_dims(conf_volume_GT[:,:,:,d], -1), refined_conf_volume[d].shape[1:3])
    #loss+=tf.reduce_sum((refined_conf_volume[d]-resized_GT_slice)**2)
#    loss+=tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(labels=resized_GT_slice, logits=refined_conf_volume[d]))
    
loss=tf.check_numerics(loss, "Nan in loss")

#definition learning setp
learning_rate = tf.placeholder(tf.float32)
optimizer = tf.train.RMSPropOptimizer(learning_rate = learning_rate)
train_op = optimizer.minimize(loss=loss)



# Initialisaiton

In [ ]:
# =============================================================================
# Initializors
# =============================================================================
tf.logging.info("Initialisaiton")

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

writer = tf.summary.FileWriter("./logs", sess.graph)

init = tf.global_variables_initializer()
sess.run(init)
sess.run(training_iterator.initializer)
train_init_op = training_iterator.make_initializer(training_set)
sess.run(train_init_op)
#img_left = sess.run(rgb_left_train)
#print img_left.shape

# =============================================================================
# Checkpoints and Visualizations
# =============================================================================
if is_from_beginning:
    lr = 1e-5
    step = 0
    val_step = 0
    losses = np.zeros((1,3))
    #loading pretrained stereonet weigts
    tf.logging.info("Restoring variables from "+stereonet_weigts)
    rv = optimistic_restore(sess, stereonet_weigts)
    tf.logging.info("Restored %d vars"%rv)
else:
    saver1 = tf.train.Saver(var_list=tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES))
    saver1.restore(sess, './checkpoints/stereoNet_testrun_9.69773729788e-07_step_88000val_0.ckpt')
    lr = 9.69773729788e-07
    step = 88000
    val_step = 0
    #9.69773729788e-07_step_88000val_0.ckpt
    
    
tf.summary.scalar('loss', loss)

tf.summary.image('GT_blur', tf.clip_by_value(blur_image_gt,0,1))
tf.summary.image('blur_image', tf.clip_by_value(blur_image,0,1))
tf.summary.image('diff_image', tf.clip_by_value(tf.abs(blur_image-blur_image_gt),0,1))

padded_conf_volume = []
for conf_slice in refined_conf_volume:
    padded_conf_volume.append(tf.image.resize_image_with_crop_or_pad(conf_slice, full_h, full_w))
gif_summary('up_refined_conf_volume', tf.cast(255*tf.stack(padded_conf_volume, 1), tf.uint8), fps=10)
gif_summary('conf_volume', tf.cast(255*tf.expand_dims(tf.transpose(conf_volume, [0,3,1,2]), -1), tf.uint8), fps=10)
gif_summary('GT_conf_volume', tf.cast(255*tf.expand_dims(tf.transpose(conf_volume_GT, [0,3,1,2]), -1), tf.uint8), fps=10)

train_summary_op = tf.summary.merge_all()

# Training

In [ ]:
n_epoch = 100
n_iter = 200000
save_n_every_step = 2000
show_n_every_step = 1000

force_show_steps = [2,4,8,16,32,64,128,256,512]

loss_values=[]

# =============================================================================
# Training loop
# =============================================================================
tf.logging.info("Starting training")

for each_epoch in range(n_epoch):
    for each_iter in range(n_iter):
        
        run_list = [train_op,
                    train_summary_op,
                    loss
                   ]
        
        output = sess.run(run_list, feed_dict={learning_rate:lr})
        
        train_summary = output[1]
        loss_out = output[2]

        if step != 0 and step % save_n_every_step == 0:
            lr *= 0.9
            saver = tf.train.Saver(var_list = tf.global_variables())
            save_path = saver.save(sess,"checkpoints/stereoNet_testrun_" + str(lr) + "_step_" + str(step) + "val_" + str(val_step) + '.ckpt')
        
        if (step != 0 and step % show_n_every_step == 0) or step in force_show_steps:
            tf.logging.info("Step %d. Loss=%f"%(step,loss_out))
            writer.add_summary(train_summary,int(step))
            
        else:
            from math import isnan
            if isnan(loss_out):
                raise BaseException("Loss in Nan at step %d"%step)
       
        step += 1

# Test


In [11]:
from PIL import Image
from stereonet.utils import readPFM

import tensorflow as tf
import os
import numpy as np
from refocus_algorithms.layered_dof_tf import layered_bluring
from blur_CV_refinement.model import refnet_cv_refinement

checkpoint_to_load = "/home/matthieu/dev/refnet/blur_CV_refinement/checkpoints/stereoNet_testrun_1.04495676332e-09_step_174000val_0.ckpt"

#params
target_disparity = [0, 50, 100, 150, 200, 250]
blur_magnitude = 0.1
max_disp = 300
min_disp = 0
downsampling_trick_max_kernel_size=11
from_scale = 1
#
full_w = 960
full_h = 540
resolution = (full_h,full_w)
#
out_folder  = "/home/matthieu/dev/refnet/blur_CV_refinement/test_out"

In [12]:

#inputs
path_to_left_rgb = '/home/matthieu/kgx_nfs2/data/external/sceneflow/frames_cleanpass/35mm_focallength/scene_mixed_test/left/'
path_to_right_rgb = '/home/matthieu/kgx_nfs2/data/external/sceneflow/frames_cleanpass/35mm_focallength/scene_mixed_test/right'
left_rgb_names_test = [os.path.join(path_to_left_rgb,each_name) for each_name in os.listdir(path_to_left_rgb)]
right_rgb_names_test = [os.path.join(path_to_right_rgb,each_name) for each_name in os.listdir(path_to_right_rgb)]
path_to_disparity = '/home/matthieu/kgx_nfs2/data/external/sceneflow/disparity/35mm_focallength/scene_mixed_test'
disparity_names_test = [os.path.join(path_to_disparity,each_name) for each_name in os.listdir(path_to_disparity)]
left_rgb_names_test.sort()
right_rgb_names_test.sort()
disparity_names_test.sort()



#Get forward 
for f in range(len(target_disparity)):
    tf.reset_default_graph()
    
    # Sizes at compile time
    rgb_left_test = tf.placeholder(shape=[1, full_h, full_w, 3], dtype=tf.float32)
    rgb_right_test = tf.placeholder(shape=[1, full_h, full_w, 3], dtype=tf.float32)
    disparity_test = tf.placeholder(shape=[1, full_h, full_w, 1], dtype=tf.float32)
    
    blur_image, _, _ = refnet_cv_refinement(rgb_left_test, rgb_right_test, 
                                               target_disparity[f], blur_magnitude,
                                               downsampling_trick_max_kernel_size=downsampling_trick_max_kernel_size,
                                               min_disp = min_disp, max_disp = max_disp,
                                               is_training = True)
    #open sess
    config = tf.ConfigProto()
    config.gpu_options.allow_growth=True
    sess = tf.Session(config=config)

    #init
    init = tf.global_variables_initializer()
    sess.run(init)

    #restore weigts 
    saver1 = tf.train.Saver(var_list=tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES))
    saver1.restore(sess, checkpoint_to_load)

    #do test
    for i in range(len(left_rgb_names_test)):
        print(i)
        rgb_left_data = np.expand_dims(np.asarray(Image.open(left_rgb_names_test[i]))/255.0, 0)
        rgb_right_data = np.expand_dims(np.asarray(Image.open(right_rgb_names_test[i]))/255.0, 0)
        disparity_data = np.expand_dims(np.expand_dims(readPFM(disparity_names_test[i]), 0), -1)

        blur_image_out = sess.run(blur_image, 
                                 feed_dict={ rgb_left_test:rgb_left_data,
                                             rgb_right_test:rgb_right_data})

        Image.fromarray((255*blur_image_out[0]).astype(np.uint8)).save(out_folder+"/%d_%d_cv_refinement.png"%(i,target_disparity[f]))



Disparity slice 8.000000
INFO:tensorflow:Bluring with radius size 0.800000 kernel size 3
INFO:tensorflow:Bluring with radius size 0.800000 kernel size 3
Disparity slice 16.000000
INFO:tensorflow:Bluring with radius size 1.600000 kernel size 5
INFO:tensorflow:Bluring with radius size 1.600000 kernel size 5
Disparity slice 24.000000
INFO:tensorflow:Bluring with radius size 2.400000 kernel size 7
INFO:tensorflow:Bluring with radius size 2.400000 kernel size 7
Disparity slice 32.000000
INFO:tensorflow:Bluring with radius size 3.200000 kernel size 9
INFO:tensorflow:Bluring with radius size 3.200000 kernel size 9
Disparity slice 40.000000
INFO:tensorflow:Bluring with radius size 4.000000 kernel size 11
INFO:tensorflow:Bluring with radius size 4.000000 kernel size 11
Disparity slice 48.000000
INFO:tensorflow:Bluring with radius size 4.800000 kernel size 11
INFO:tensorflow:Bluring with radius size 4.800000 kernel size 11
Disparity slice 56.000000
INFO:tensorflow:Bluring with radius size 4.7384

Disparity slice 120.000000
INFO:tensorflow:Bluring with radius size 2.000000 kernel size 7
INFO:tensorflow:Bluring with radius size 2.000000 kernel size 7
Disparity slice 128.000000
INFO:tensorflow:Bluring with radius size 2.800000 kernel size 7
INFO:tensorflow:Bluring with radius size 2.800000 kernel size 7
Disparity slice 136.000000
INFO:tensorflow:Bluring with radius size 3.600000 kernel size 9
INFO:tensorflow:Bluring with radius size 3.600000 kernel size 9
Disparity slice 144.000000
INFO:tensorflow:Bluring with radius size 4.400000 kernel size 11
INFO:tensorflow:Bluring with radius size 4.400000 kernel size 11
INFO:tensorflow:Image composition
INFO:tensorflow:Restoring parameters from /home/matthieu/dev/refnet/blur_CV_refinement/checkpoints/stereoNet_testrun_1.04495676332e-09_step_174000val_0.ckpt
0
1
2
3
4
5
6
7
8
9
10
Disparity slice 8.000000
INFO:tensorflow:Bluring with radius size 5.038710 kernel size 13
INFO:tensorflow:Bluring with radius size 5.038710 kernel size 13
Disparity

INFO:tensorflow:Bluring with radius size 5.291892 kernel size 13
Disparity slice 80.000000
INFO:tensorflow:Bluring with radius size 5.054054 kernel size 13
INFO:tensorflow:Bluring with radius size 5.054054 kernel size 13
Disparity slice 88.000000
INFO:tensorflow:Bluring with radius size 5.091429 kernel size 13
INFO:tensorflow:Bluring with radius size 5.091429 kernel size 13
Disparity slice 96.000000
INFO:tensorflow:Bluring with radius size 5.133333 kernel size 13
INFO:tensorflow:Bluring with radius size 5.133333 kernel size 13
Disparity slice 104.000000
INFO:tensorflow:Bluring with radius size 5.180645 kernel size 13
INFO:tensorflow:Bluring with radius size 5.180645 kernel size 13
Disparity slice 112.000000
INFO:tensorflow:Bluring with radius size 5.234483 kernel size 13
INFO:tensorflow:Bluring with radius size 5.234483 kernel size 13
Disparity slice 120.000000
INFO:tensorflow:Bluring with radius size 4.931034 kernel size 11
INFO:tensorflow:Bluring with radius size 4.931034 kernel size